In [16]:
import geopandas, pandas, numpy
from string import ascii_lowercase as alc

In [17]:
"""
#run this code snippet when you are running this code for the first time, this takes in the DA files from 2006 and 2021
#It selects only Ontario DAs and export it as csv, which makes re-running the code in the future MUCH faster


print("canada_da_21")
canada_da_21 = geopandas.read_file("../census-file-handling/Data/lda_000b21a_e/lda_000b21a_e.shp")
#print("canada_da_06")
canada_da_06 = geopandas.read_file("../census-file-handling/Data/gda_000b06a_e/gda_000b06a_e.shp")
print("canada_csd_21")
canada_csd_21 = geopandas.read_file("../census-file-handling/Data/lcsd000b21a_e/lcsd000b21a_e.shp")

ontario_da_21 = canada_da_21[canada_da_21['PRUID'] == '35']
ontario_da_06 = canada_da_06[canada_da_06['PRUID'] == '35']
ontario_csd_21 = canada_csd_21[canada_csd_21['PRUID'] == '35']


ontario_da_21.to_csv("../census-file-handling/Data/Census Data/ontario_da_21.csv")
ontario_da_06.to_csv("../census-file-handling/Data/Census Data/ontario_da_06.csv")
ontario_csd_21.to_csv("../census-file-handling/Data/Census Data/ontario_csd_21.csv")"""

'\nprint("canada_da_21")\ncanada_da_21 = geopandas.read_file("../census-file-handling/Data/lda_000b21a_e/lda_000b21a_e.shp")\n#print("canada_da_06")\ncanada_da_06 = geopandas.read_file("../census-file-handling/Data/gda_000b06a_e/gda_000b06a_e.shp")\nprint("canada_csd_21")\ncanada_csd_21 = geopandas.read_file("../census-file-handling/Data/lcsd000b21a_e/lcsd000b21a_e.shp")\n\nontario_da_21 = canada_da_21[canada_da_21[\'PRUID\'] == \'35\']\nontario_da_06 = canada_da_06[canada_da_06[\'PRUID\'] == \'35\']\nontario_csd_21 = canada_csd_21[canada_csd_21[\'PRUID\'] == \'35\']\n\n\nontario_da_21.to_csv("../census-file-handling/Data/Census Data/ontario_da_21.csv")\nontario_da_06.to_csv("../census-file-handling/Data/Census Data/ontario_da_06.csv")\nontario_csd_21.to_csv("../census-file-handling/Data/Census Data/ontario_csd_21.csv")'

In [18]:
ontario_da_21 = pandas.read_csv("../census-file-handling/Data/Census Data/ontario_da_21.csv")
ontario_da_06 = pandas.read_csv("../census-file-handling/Data/Census Data/ontario_da_06.csv")
ontario_csd_21 = pandas.read_csv("../census-file-handling/Data/Census Data/ontario_csd_21.csv")

ontario_da_21 = ontario_da_21.drop(['Unnamed: 0', 'geometry'], axis = 1)
ontario_da_06 = ontario_da_06.drop(['Unnamed: 0', 'geometry'], axis = 1)
ontario_csd_21 = ontario_csd_21.drop(['Unnamed: 0', 'geometry'], axis = 1)

ontario_da_21['DAUID'] = ontario_da_21['DAUID'].astype('str')
ontario_da_06['CSDUID'] = ontario_da_06['CSDUID'].astype('str')
ontario_csd_21['CSDUID'] = ontario_csd_21['CSDUID'].astype('str')

Bridging the Gap Between CSDUID and DAUID


In [20]:

#import census reference table, which contains the DGUID for DA and CSD level. 
region_summary = pandas.read_csv("../regions-summary.csv")
DA_to_CSD = pandas.read_csv("../census-file-handling/Data/2021_DA_CSDUID_Reference_Table.csv")

#merge the reference table to the Ontario DA file for 2021
DA_with_CSDDGUID = pandas.merge(ontario_da_21, DA_to_CSD, left_on = "DGUID", right_on = "DADGUID_ADIDUGD", sort=False)
#copy the 'DGUID', 'CSDUID' to a new dataframe called DA_with_CSD, then merge it to DA_with_CSDDGUID
DA_with_CSD = ontario_csd_21[['DGUID', 'CSDUID']].copy()
GGH_DA_with_CSDUID_21 = pandas.merge(DA_with_CSDDGUID, DA_with_CSD,  left_on= "CSDDGUID_SDRIDUGD", right_on= "DGUID", sort=False)
#remove redundant fields 
GGH_DA_with_CSDUID_21 = GGH_DA_with_CSDUID_21.drop(["CSDDGUID_SDRIDUGD", "LANDAREA", "DADGUID_ADIDUGD", "DGUID_y","PRUID", "DGUID_x"], axis = 1)
GGH_DA_with_CSDUID_06 = ontario_da_06.drop(["CDUID", "ERUID", "CCSUID", "CTUID", "CMAUID", "PRUID"], axis = 1)

#turn the region summary into a list
censusIDList = region_summary['CDUID2'].tolist()
municipalityList = region_summary['Municipality'].tolist()
macroList = region_summary["Macro"].tolist()
#loop through the list to add a new field called CSDID2, CSDID2 will be a new key for joining with regions-summary.csv
i = 0
while i < len(censusIDList):
    regionID = censusIDList[i]
    macro = macroList[i]
    municipality = municipalityList[i]
    GGH_DA_with_CSDUID_21.loc[GGH_DA_with_CSDUID_21['CSDUID'].str.startswith(f'{regionID}'), "Macro"] = macro
    GGH_DA_with_CSDUID_06.loc[GGH_DA_with_CSDUID_06['CSDUID'].str.startswith(f'{regionID}'), "Macro"] = macro
    GGH_DA_with_CSDUID_21.loc[GGH_DA_with_CSDUID_21['CSDUID'].str.startswith(f'{regionID}'), "Municipality"] = municipality
    GGH_DA_with_CSDUID_06.loc[GGH_DA_with_CSDUID_06['CSDUID'].str.startswith(f'{regionID}'), "Municipality"] = municipality
    i+=1

GGH_DA_with_CSDUID_21 = GGH_DA_with_CSDUID_21.dropna()
GGH_DA_with_CSDUID_06 = GGH_DA_with_CSDUID_06.dropna()
GGH_DA_with_CSDUID_21.to_csv("../census-file-handling/Data/GGH_DA_with_CSDUID_21.csv")
GGH_DA_with_CSDUID_06.to_csv("../census-file-handling/Data/GGH_DA_with_CSDUID_06.csv")



Joining Census Data With the Census ID Converstion Table

In [21]:
import geopandas, pandas, numpy
from string import ascii_lowercase as alc

census_ontario_data_06 = pandas.read_csv("../census-file-handling/Data/Census Data/Census2006.csv", encoding='utf-8')
census_ontario_data_21 = pandas.read_csv("../census-file-handling/Data/Census Data/Census2021.csv", encoding='utf-8')


In [22]:
#merge the DA with CSD UID with census data
GGH_DA_with_CSDUID_06["DAUID"] = GGH_DA_with_CSDUID_06["DAUID"].astype('str')
census_ontario_data_06["Dauid"] = census_ontario_data_06["Dauid"].astype('str')
GGH_Census_06 = pandas.merge(GGH_DA_with_CSDUID_06, census_ontario_data_06,  left_on= "DAUID", right_on= "Dauid", sort=False)

GGH_DA_with_CSDUID_21["DAUID"] = GGH_DA_with_CSDUID_21["DAUID"].astype('str')
census_ontario_data_21["GEO UID"] = census_ontario_data_21["GEO UID"].astype('str')
GGH_Census_21 = pandas.merge(GGH_DA_with_CSDUID_21, census_ontario_data_21,  left_on= "DAUID", right_on= "GEO UID", sort=False)

#lose some unnecessary fields
GGH_Census_06 = GGH_Census_06.drop('Dauid', axis =1)
GGH_Census_21 = GGH_Census_21.drop(['GEO UID', 'Land area in square kilometres'], axis = 1)

fields_06 = GGH_Census_06.columns [4:].tolist()
fields_21 = GGH_Census_21.columns [4:].tolist()

#add alphabets into the field names to keep the columns in place, merging and pivoting information would rearrange the fields.
#also differentiate 2006 and 2021 fields. 
i = 0
while i < len(fields_06):
    field_06 = fields_06[i].replace("building that has ", "").replace(",","").replace("-", " ").replace(" in a","")
    field_21 = fields_21[i].replace("building that has ", "").replace(",","").replace("-", " ").replace(" in a","")

    GGH_Census_06.rename(columns={f'{fields_06[i]}': f'2006-{alc[i]}-{field_06}'}, inplace=True)
    GGH_Census_21.rename(columns={f'{fields_21[i]}': f'2021-{alc[i]}-{field_21}'}, inplace=True)
    i+=1

#GGH Census Data Exported to CSV
GGH_Census_06.to_csv("../census-summary-data-2006-2021/part-c-results/GGH-Census-da-06.csv")
GGH_Census_21.to_csv("../census-summary-data-2006-2021/part-c-results/GGH-Census-da-21.csv")


In [51]:
#PIVOTTTTT!!! PIVOTTT!!!
#pivot table to summarize information to the CSD level. 
GGH06_fields = GGH_Census_06.columns [4:].tolist() #the fields we want to summarize
GGH21_fields = GGH_Census_21.columns [4:].tolist() #the fields we want to summarize

pivot_table_06 = pandas.pivot_table(data = GGH_Census_06, values = GGH06_fields, index = ['Municipality'], aggfunc = numpy.sum)
pivot_table_21 = pandas.pivot_table(data = GGH_Census_21, values = GGH21_fields, index = ['Municipality'], aggfunc = numpy.sum)
pivot_table_06 = pivot_table_06.reset_index() #turn it back into a dataframe
pivot_table_21 = pivot_table_21.reset_index()

#join 2006 and 2021 table together. 
pivot_table = pandas.merge(pivot_table_06, pivot_table_21 , on = "Municipality", sort=False)
#pivot_table = pivot_table.drop(pivot_table.columns.tolist()[13:15], axis =1)
pivot_field = pivot_table.columns.tolist()[1:]
i = 0
#calculate the change between 2006 and 2021
while i <= (len(pivot_field)-1)/2:
    #feed 2006's census field name for the "change" section
    changeField = pivot_field[i].replace("2006", "Chg") 
    
    #create the change field by subtracting 2006 information from 2021 information. 
    pivot_table[f'{changeField}'] = pivot_table [pivot_field[i+12]] - pivot_table [pivot_field[i]] 
    i+=1

#rename the column, remove the alphabets added to keep the columns in place. 
i=1
column = pivot_table.columns.tolist()
while i < len(column):
    columnList = column[i].split("-")
    pivot_table = pivot_table.rename(columns={f"{column[i]}": f"{columnList[0]} {columnList[2]}"})
    
    i+=1

pivot_table.to_csv("../census-summary-data-2006-2021/part-c-results/GGH-census-region-summary-2006-2021.csv")


Chg-a-Population
Chg-b-Total Private dwellings
Chg-c-Total dwellings structures
Chg-d-Single detached house
Chg-e-Semi detached house
Chg-f-Row house
Chg-g-Apartment duplex
Chg-h-Apartment five or more storeys
Chg-i-Apartment fewer than five storeys
Chg-j-Other single attached house
Chg-k-Movable dwelling
Chg-l-Total households
next section
{'2006-a-Population': '2006 Population'}
{'2006-b-Total Private dwellings': '2006 Total Private dwellings'}
{'2006-c-Total dwellings structures': '2006 Total dwellings structures'}
{'2006-d-Single detached house': '2006 Single detached house'}
{'2006-e-Semi detached house': '2006 Semi detached house'}
{'2006-f-Row house': '2006 Row house'}
{'2006-g-Apartment duplex': '2006 Apartment duplex'}
{'2006-h-Apartment five or more storeys': '2006 Apartment five or more storeys'}
{'2006-i-Apartment fewer than five storeys': '2006 Apartment fewer than five storeys'}
{'2006-j-Other single attached house': '2006 Other single attached house'}
{'2006-k-Movable dw

In [ ]:
ggh_regions = geopandas.read_file("../ggh-growth-plan-regions.geojson")
ggh1 = pandas.merge(ggh_regions, pivot_table, right_on = "Municipality", left_on = "Name", sort=False).drop(['Name'], axis = 1)
ggh1.explore(column = 'Chg Apartment five or more storeys', legend = True, cmap = "Purples", scheme="quantiles")